In [1]:
import numpy as np
from tensorflow.keras import datasets, layers, models

import tvm
from tvm import te
import tvm.relay as relay
from tvm.contrib.download import download_testdata
import keras
import numpy as np
import tensorflow as tf
from PIL import Image
from tvm.contrib.download import download_testdata
from SimpleModelKeras import Net

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import keras


# def Net(shape):
#     input_layer = layers.Input(shape=shape)
#     out = layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same')(input_layer)
#     out = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(out)
#     out = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(out)
#     out = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(out)
#     out = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(out)
#     out = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(out)
#     out = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(out)
#     out = layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')(out)
#     model = keras.models.Model(inputs=input_layer, outputs=out)
#     return model

In [2]:
img_url = "https://github.com/dmlc/mxnet.js/blob/main/data/cat.png?raw=true"
img_path = download_testdata(img_url, "cat.png", module="data")
img = Image.open(img_path).resize((256,256))

model = Net((256,256,3))
data = np.array(img)[np.newaxis, :].astype("float32").transpose([0, 3, 1, 2])

shape_dict = {"input_1": data.shape}
mod, params = relay.frontend.from_keras(model, shape_dict)
print(mod.astext(show_meta_data=False))

# compile the model
target = "cuda"
dev = tvm.cuda(0)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

File /home/j/.tvm_test_data/data/cat.png exists, skip.
#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
}



One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


executor : {
  "nodes": [
    {
      "op": "null", 
      "name": "input_1", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "p0", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "p1", 
      "inputs": []
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_default_fused_nn_conv2d_add", 
      "attrs": {
        "num_outputs": "1", 
        "flatten_data": "0", 
        "data_layout": "NCHW", 
        "hash": "7c04a9dce6fc0107", 
        "num_inputs": "3", 
        "kernel_layout": "OIHW", 
        "out_layout": "", 
        "func_name": "tvmgen_default_fused_nn_conv2d_add"
      }, 
      "inputs": [
        [
          0, 
          0, 
          0
        ], 
        [
          1, 
          0, 
          0
        ], 
        [
          2, 
          0, 
          0
        ]
      ]
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_default_fused_nn_max_pool2d", 
      "attrs": {
        "num_outputs": "1", 
        "layou

In [3]:
from os import dup
from sys import displayhook
import pygraphviz as pgv
import re
import json

In [4]:
json_data = json.loads(lib.get_graph_json())

In [6]:
A = pgv.AGraph(directed=True)
for node in json_data['nodes']:
    for src in node['inputs']:
        A.add_edge(json_data['nodes'][src[0]]['name'], node['name'])
    # dst = digraph[node]['name']
    # srcs = digraph[node]['srcs']
    # A.add_node(dst, shape='box')
    # for src in srcs:
        # try:
            # A.add_edge(digraph[src]['name'], dst)
        # except:
            # A.add_edge(src, dst)
# # A.draw("graph_before.png", prog='dot')
A.draw("simple_model_before_fuse.png", prog='dot')


In [28]:
json_data

{'nodes': [{'op': 'null', 'name': 'input_1', 'inputs': []},
  {'op': 'null', 'name': 'p0', 'inputs': []},
  {'op': 'tvm_op',
   'name': 'tvmgen_default_fused_nn_conv2d',
   'attrs': {'num_outputs': '1',
    'flatten_data': '0',
    'out_layout': '',
    'func_name': 'tvmgen_default_fused_nn_conv2d',
    'hash': '127ca854e2dbdd9e',
    'num_inputs': '2',
    'kernel_layout': 'OIHW',
    'data_layout': 'NCHW'},
   'inputs': [[0, 0, 0], [1, 0, 0]]},
  {'op': 'null', 'name': 'p1', 'inputs': []},
  {'op': 'tvm_op',
   'name': 'tvmgen_default_fused_add',
   'attrs': {'num_inputs': '2',
    'num_outputs': '1',
    'hash': '2ef207a3d2c64a2f',
    'flatten_data': '0',
    'func_name': 'tvmgen_default_fused_add'},
   'inputs': [[2, 0, 0], [3, 0, 0]]},
  {'op': 'tvm_op',
   'name': 'tvmgen_default_fused_nn_max_pool2d',
   'attrs': {'num_outputs': '1',
    'layout': 'NCHW',
    'func_name': 'tvmgen_default_fused_nn_max_pool2d',
    'flatten_data': '0',
    'hash': '6a1842ec1de4993d',
    'num_inpu

In [27]:
for k in json_data:
    print(k)

nodes
arg_nodes
heads
attrs
node_row_ptr


In [18]:
lib.target

{2: cuda -keys=cuda,gpu -max_num_threads=1024 -thread_warp_size=32}

In [7]:
# print(lib.get_graph_json())
print(lib.get_executor_config())

{
  "nodes": [
    {
      "op": "null", 
      "name": "input_1", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "p0", 
      "inputs": []
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_default_fused_nn_conv2d", 
      "attrs": {
        "num_outputs": "1", 
        "flatten_data": "0", 
        "out_layout": "", 
        "func_name": "tvmgen_default_fused_nn_conv2d", 
        "hash": "127ca854e2dbdd9e", 
        "num_inputs": "2", 
        "kernel_layout": "OIHW", 
        "data_layout": "NCHW"
      }, 
      "inputs": [
        [
          0, 
          0, 
          0
        ], 
        [
          1, 
          0, 
          0
        ]
      ]
    }, 
    {
      "op": "null", 
      "name": "p1", 
      "inputs": []
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_default_fused_add", 
      "attrs": {
        "num_inputs": "2", 
        "num_outputs": "1", 
        "hash": "2ef207a3d2c64a2f", 
        "flatten_data": "0", 
        "f

In [4]:
import json

In [5]:
ll = json.loads(graph_json)

In [19]:
for i in ll['nodes']:
    print(i)

{'op': 'null', 'name': 'data', 'inputs': []}
{'op': 'null', 'name': 'p0', 'inputs': []}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_nn_conv2d', 'attrs': {'num_outputs': '1', 'flatten_data': '0', 'data_layout': 'NCHW', 'hash': 'fcf4c3eba41f0b65', 'num_inputs': '2', 'kernel_layout': 'OIHW', 'out_layout': '', 'func_name': 'tvmgen_default_fused_nn_conv2d'}, 'inputs': [[0, 0, 0], [1, 0, 0]]}
{'op': 'null', 'name': 'p1', 'inputs': []}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_add', 'attrs': {'num_inputs': '2', 'num_outputs': '1', 'hash': '9944b64f2bf8d828', 'flatten_data': '0', 'func_name': 'tvmgen_default_fused_add'}, 'inputs': [[2, 0, 0], [3, 0, 0]]}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_nn_relu', 'attrs': {'num_inputs': '1', 'num_outputs': '1', 'hash': 'ee5a8a7342ec30c9', 'flatten_data': '0', 'func_name': 'tvmgen_default_fused_nn_relu'}, 'inputs': [[4, 0, 0]]}
{'op': 'null', 'name': 'p2', 'inputs': []}
{'op': 'tvm_op', 'name': 'tvmgen_default_fused_nn_contrib_conv2d_winog

In [5]:
out.flatten()[0:10]

array([0.00100052, 0.00099585, 0.00099843, 0.00100801, 0.00099786,
       0.00100267, 0.00100183, 0.00100203, 0.00099908, 0.0010008 ],
      dtype=float32)

In [6]:
# print first 10 elements of output
print(out.flatten()[0:10])

from tvm.contrib import utils

temp = utils.tempdir()
path_lib = temp.relpath('deploy_lib.tar')
lib.export_library(path_lib)
print(temp.listdir())

# load the module back.
loaded_lib = tvm.runtime.load_module(path_lib)
input_data = tvm.nd.array(data)

module = graph_executor.GraphModule(loaded_lib['default'](dev))
module.run(data=input_data)
out_deploy = module.get_output(0).asnumpy()

# print first 10 elements of output
print(out_deploy.flatten()[0:10])

# check whether the output from deployed module is consistent with original one
tvm.testing.assert_allclose(out_deploy, out, atol=1e-5)


[0.00100052 0.00099585 0.00099843 0.00100801 0.00099786 0.00100267
 0.00100183 0.00100203 0.00099908 0.0010008 ]
['deploy_lib.tar']
[0.00100052 0.00099585 0.00099843 0.00100801 0.00099786 0.00100267
 0.00100183 0.00100203 0.00099908 0.0010008 ]
